In [1]:
import numpy as np
from PIL import Image

In [2]:
def padding(image: np.ndarray) -> np.ndarray:
    padded_image = np.zeros((image.shape[0] + 2, image.shape[1] + 2, image.shape[2]), dtype=image.dtype)
    padded_image[1:-1, 1:-1, :] = image
    padded_image[0, 1:-1] = image[1, :]
    padded_image[-1, 1:-1] = image[-2, :]
    padded_image[1:-1, 0] = image[:, 1]
    padded_image[1:-1, -1] = image[:, -2]
    padded_image[0, 0] = image[1, 1]
    padded_image[0, -1] = image[1, -2]
    padded_image[-1, 0] = image[-2, 1]
    padded_image[-1, -1] = image[-2, -2]

    return padded_image

In [3]:
def check_input(image: Image):
    if image.mode != 'RGB':
        raise ValueError('Input image must be RGB')

In [4]:
input_image = Image.open("images/medium.png")
check_input(input_image)
np_input_image = np.array(input_image, dtype=np.float32)
np_input_image = padding(np_input_image)

print("Input image shape: ", np_input_image.shape, "Input image dtype: ", np_input_image.dtype)

Input image shape:  (502, 502, 3) Input image dtype:  float32


In [5]:
bayer_pattern = np.array([[[0.,1.], [0.,0.]], 
                          [[1.,0.], [0.,1.]], 
                          [[0.,0.], [1.,0.]]]).transpose(1, 2, 0).astype(np.float32)

pil_bayer_pattern = Image.fromarray((bayer_pattern * 255).astype(np.uint8))
pil_bayer_pattern.save("bayer_pattern.png")
print("Bayer Pattern Shape:", bayer_pattern.shape, "Bayer Pattern Dtype:",bayer_pattern.dtype)

Bayer Pattern Shape: (2, 2, 3) Bayer Pattern Dtype: float32


In [6]:
def mosaicking(input_image, bayer):
    output_image = np.zeros_like(input_image)
    for i in range(1, input_image.shape[0] - 1, 2):
        for j in range(1, input_image.shape[1] - 1, 2):
            output_image[i:i+2, j:j+2] = input_image[i:i+2, j:j+2] * bayer
    return output_image

In [7]:
def de_mosaicking(input_image):
    output_image = input_image.copy()

    for i in range(0, input_image.shape[0]-2):
        for j in range(0, input_image.shape[1]-2):
            if input_image[i+1, j+1, 0] == 0:
                red_channel = input_image[i:i+3, j:j+3,0]
                if np.count_nonzero(red_channel) == 0:
                    red_channel_avg = 0
                else:
                    red_channel_avg = np.sum(red_channel) / np.count_nonzero(red_channel)
                output_image[i+1, j+1, 0] = red_channel_avg
            
            if input_image[i+1, j+1, 1] == 0:
                green_channel = input_image[i:i+3, j:j+3,1]
                if np.count_nonzero(green_channel) == 0:
                    green_channel_avg = 0
                else:
                    green_channel_avg = np.sum(green_channel) / np.count_nonzero(green_channel)
                output_image[i+1, j+1, 1] = green_channel_avg

            if input_image[i+1, j+1, 2] == 0:
                blue_channel = input_image[i:i+3, j:j+3,2]
                if np.count_nonzero(blue_channel) == 0:
                    blue_channel_avg = 0
                else:
                    blue_channel_avg = np.sum(blue_channel) / np.count_nonzero(blue_channel)
                output_image[i+1, j+1, 2] = blue_channel_avg

    output_image = output_image[1:-1, 1:-1]

    return output_image

In [8]:
np_mosaic_image = mosaicking(np_input_image, bayer_pattern)
print("Mosaic Image Shape:", np_mosaic_image.shape, "Mosaic Image Dtype:",np_mosaic_image.dtype)
print("Input:", np_input_image[10][1], "Output:", np_mosaic_image[10][1])

pil_mosaic_image = Image.fromarray(np_mosaic_image.astype(np.uint8))
pil_mosaic_image.save("mosaic_image.png")

Mosaic Image Shape: (502, 502, 3) Mosaic Image Dtype: float32
Input: [125. 162. 154.] Output: [  0.   0. 154.]


In [9]:
np_output_image = de_mosaicking(np_mosaic_image)
pil_output_image = Image.fromarray(np_output_image.astype(np.uint8))
pil_output_image.save("output_image.png")

In [10]:
def mse(image1, image2):
    for c in range(3):
        for i in range(3):
            for j in range(3):
                squared_diff = (image1[i, j, c] - image2[i, j, c]) ** 2
                mse = np.sum(squared_diff) / (image1.shape[0] * image1.shape[1] * image1.shape[2])
    return mse

def psnr(image1, image2):
    mse_val = mse(image1, image2)
    psnr = 10 * np.log10(255**2 / mse_val)
    return psnr

def mae(image1, image2):
    for c in range(3):
        for i in range(3):
            for j in range(3):
                abs_diff = np.abs(image1[i, j, c] - image2[i, j, c])
                mae = np.sum(abs_diff) / (image1.shape[0] * image1.shape[1] * image1.shape[2])

    return mae

print("MSE:", mse(np_input_image, np_output_image))
print("PSNR:", psnr(np_input_image, np_output_image))
print("MAE:", mae(np_input_image, np_output_image))

MSE: 0.0
PSNR: inf
MAE: 0.0


/var/folders/4f/c42dygvd2154cwkfn2wt8ztw0000gn/T/ipykernel_3071/1976597481.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  psnr = 10 * np.log10(255**2 / mse_val)
